In [1]:
from IPython.display import clear_output
%cd /content/drive/MyDrive/collab_sandbox/learn_DL/dive_into_dl/
# !pip install colabcode
clear_output()

In [4]:
import pandas as pd
import torch
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn.apionly as sns
from sklearn.model_selection import KFold


In [ ]:
dataset = pd.read_csv("./data/HousePricesKaggle/train.csv")
dataset_train, dataset_test = train_test_split(dataset, test_size=0.20, random_state=42)

dataset_train = dataset_train.reset_index()
dataset_test = dataset_test.reset_index()

drop_columns = 'index Id'.split()
dataset_train = dataset_train.drop(columns=drop_columns)
dataset_test = dataset_test.drop(columns=drop_columns)

# # select_columns = "SalePrice LotFrontage	LotArea OverallQual	OverallCond YearBuilt	YearRemodAdd MasVnrArea BsmtFinSF1 BsmtFinSF2	BsmtUnfSF TotalBsmtSF 1stFlrSF	2ndFlrSF	LowQualFinSF	GrLivArea	BsmtFullBath	BsmtHalfBath FullBath	HalfBath	BedroomAbvGr	KitchenAbvGr TotRmsAbvGrd Fireplaces GarageYrBlt GarageCars	GarageArea WoodDeckSF	OpenPorchSF	EnclosedPorch	3SsnPorch	ScreenPorch	PoolArea MiscVal MoSold YrSold MSSubClass".split()
# # select_columns = "SalePrice OverallQual BsmtFullBath FullBath HalfBath GarageCars".split()
select_columns = dataset_train.dtypes[dataset_train.dtypes != 'object'].index.tolist()
select_columns = list(set(select_columns) - set(['SalePrice']))

dataset_train[select_columns] = dataset_train[select_columns].apply(
    lambda x: (x - x.mean()) / (x.std()))
dataset_test[select_columns] = dataset_test[select_columns].apply(
    lambda x: (x - x.mean()) / (x.std()))

dataset_train[select_columns] = dataset_train[select_columns].fillna(dataset_train[select_columns].mean() / dataset_train[select_columns].std())
dataset_test[select_columns] = dataset_test[select_columns].fillna(dataset_test[select_columns].mean() / dataset_test[select_columns].std())

# dataset_train = pd.get_dummies(dataset_train, dummy_na=True)
# dataset_test = pd.get_dummies(dataset_test, dummy_na=True)

dataset_train.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,-0.866393,RL,-0.013811,-0.212805,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,-0.820093,0.372058,-0.455274,-1.345487,Gable,CompShg,MetalSd,MetalSd,None,-0.599726,TA,Gd,CBlock,TA,TA,No,Rec,1.036824,Unf,-0.285382,-0.400110,0.572367,GasA,TA,Y,SBrkr,0.374075,-0.801580,-0.118948,-0.406919,1.104846,-0.242766,-1.055114,-0.763770,0.13616,-0.212666,TA,-0.964153,Typ,-0.958182,NaN,Attchd,-8.759833e-01,RFn,-1.056091,-0.863467,TA,TA,Y,1.187893,-0.714046,-0.351770,-0.120956,-0.27572,-0.070962,NaN,NaN,NaN,-0.092701,-0.133360,1.649359,WD,Normal,145000
1,0.074078,RL,-0.455631,-0.265131,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,-0.088896,1.268065,0.718301,0.439026,Gable,CompShg,VinylSd,VinylSd,None,-0.599726,Gd,TA,PConc,Gd,TA,No,Unf,-0.971580,Unf,-0.285382,0.511701,-0.596291,GasA,Gd,Y,SBrkr,-0.957792,0.954679,-0.118948,0.083135,-0.818344,-0.242766,0.773333,1.236417,0.13616,-0.212666,TA,0.270639,Typ,0.591896,TA,Attchd,5.798010e-01,RFn,0.294965,-0.456069,TA,TA,Y,-0.739841,-0.137956,-0.351770,-0.120956,-0.27572,-0.070962,NaN,NaN,NaN,-0.092701,-0.507792,0.893295,WD,Normal,178000
2,-0.631275,RL,-0.134307,-0.177765,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Feedr,Norm,1Fam,1Story,-0.820093,1.268065,-1.987442,-1.683097,Gable,CompShg,MetalSd,Wd Sdng,None,-0.599726,TA,TA,CBlock,Fa,TA,No,Unf,-0.971580,Unf,-0.285382,0.504980,-0.603099,GasA,Gd,Y,FuseA,-0.965550,-0.801580,-0.118948,-1.394653,-0.818344,-0.242766,-1.055114,-0.763770,-1.10630,-0.212666,TA,-1.581549,Typ,-0.958182,NaN,NaN,-2.955365e-15,NaN,-2.407148,-2.256203,NaN,NaN,P,1.789347,-0.714046,2.289845,-0.120956,-0.27572,-0.070962,NaN,MnPrv,NaN,-0.092701,-0.507792,0.137231,WD,Normal,85000
3,-0.161040,RL,-0.415465,-0.324335,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Feedr,Norm,1Fam,1.5Fin,-0.820093,1.268065,-1.107260,-1.683097,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,0.856650,TA,TA,BrkTil,Gd,TA,No,ALQ,0.267880,Unf,-0.285382,-0.915384,-0.750599,GasA,Ex,Y,SBrkr,-0.487113,0.988803,-0.118948,0.458779,1.104846,-0.242766,-1.055114,1.236417,0.13616,-0.212666,Gd,0.270639,Typ,2.141974,TA,Detchd,-1.603875e+00,Unf,-1.056091,-1.119276,TA,TA,Y,-0.739841,-0.714046,3.900586,-0.120956,-0.27572,-0.070962,NaN,MnPrv,NaN,-0.092701,-0.133360,-0.618833,WD,Normal,175000
4,-0.161040,RL,-0.817120,-0.528808,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,1.5Fin,-0.820093,0.372058,-1.531051,-1.683097,Gable,CompShg,BrkFace,Wd Sdng,None,-0.599726,TA,TA,BrkTil,TA,TA,No,LwQ,-0.496707,Unf,-0.285382,0.531863,-0.081174,GasA,TA,Y,SBrkr,-0.370736,0.711260,-0.118948,0.311953,-0.818344,-0.242766,0.773333,-0.763770,0.13616,-0.212666,Gd,-0.346757,Typ,0.591896,Gd,Detchd,-2.210452e+00,Unf,-1.056091,-0.797147,TA,TA,Y,-0.739841,-0.714046,3.546223,-0.120956,-0.27572,-0.070962,NaN,NaN,NaN,-0.092701,-0.507792,1.649359,WD,Normal,127000


In [ ]:
# fig, ax = plt.subplots(figsize=(40,40))
# corr = dataset_train.corr()# plot the heatmap
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True), ax=ax)

In [110]:
def encode_categorical(dataset, columns, full_dataset):
  dataset = dataset.copy(deep=True)
  # print(dataset)
  dataset_cols = dataset.columns.values
  for column in columns:
    if column in dataset_cols:
      values_list = full_dataset[column].values.tolist()
      list_columns = sorted(list(map(str, list(set(values_list)))))
      list_columns_renamed = [f'{column}_{item}' for item in list_columns]
      dict_columns = {item: i for (i, item) in enumerate(list_columns)}
      
      # print(dataset[column].)
      dataset_values = dataset[column].tolist()
      for i, value in enumerate(dataset_values):
        value = str(value)

        ohe_hot = [0 for _ in range(len(list_columns))]
        if dict_columns.get(value):
          ohe_hot[dict_columns.get(value)] = 1

        # slow operation!
        dataset.loc[i, list_columns_renamed] = ohe_hot
      dataset = dataset.drop(columns=[column],) 
  
  return dataset

In [ ]:
dataset_train.dtypes[dataset_train.dtypes == 'object'].index.tolist()

In [ ]:
categorical_labels = dataset_train.dtypes[dataset_train.dtypes == 'object'].index.tolist() #list(set('MSSubClass PavedDrive MSZoning Street	Alley	LotShape	LandContour	Utilities	LotConfig	LandSlope	Neighborhood	Condition1	Condition2	BldgType	HouseStyle RoofStyle	RoofMatl	Exterior1st	Exterior2nd	MasVnrType ExterQual	ExterCond	Foundation	BsmtQual	BsmtCond	BsmtExposure	BsmtFinType1 BsmtFinType2 Heating	HeatingQC	CentralAir	Electrical KitchenQual Functional FireplaceQu	GarageType GarageFinish PoolQC	Fence	MiscFeature SaleType	SaleCondition GarageCond GarageQual GarageType GarageFinish	GarageQual GarageCond'.split()))
dataset_train = encode_categorical(dataset_train, categorical_labels, dataset)
dataset_test = encode_categorical(dataset_test, categorical_labels, dataset)
dataset_train.head(5)

In [ ]:
len(dataset_train.columns) == len(dataset_test.columns), len(dataset_test.columns), len(dataset_train.columns)

(True, 305, 305)

In [ ]:
dataset_train.head(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,GarageQual_TA,GarageQual_nan,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_nan,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_nan,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_nan,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MiscFeature_nan,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.866393,-0.013811,-0.212805,-0.820093,0.372058,-0.455274,-1.345487,-0.599726,1.036824,-0.285382,-0.400110,0.572367,0.374075,-0.801580,-0.118948,-0.406919,1.104846,-0.242766,-1.055114,-0.763770,0.13616,-0.212666,-0.964153,-0.958182,-8.759833e-01,-1.056091,-0.863467,1.187893,-0.714046,-0.351770,-0.120956,-0.27572,-0.070962,-0.092701,-0.133360,1.649359,145000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.074078,-0.455631,-0.265131,-0.088896,1.268065,0.718301,0.439026,-0.599726,-0.971580,-0.285382,0.511701,-0.596291,-0.957792,0.954679,-0.118948,0.083135,-0.818344,-0.242766,0.773333,1.236417,0.13616,-0.212666,0.270639,0.591896,5.798010e-01,0.294965,-0.456069,-0.739841,-0.137956,-0.351770,-0.120956,-0.27572,-0.070962,-0.092701,-0.507792,0.893295,178000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.631275,-0.134307,-0.177765,-0.820093,1.268065,-1.987442,-1.683097,-0.599726,-0.971580,-0.285382,0.504980,-0.603099,-0.965550,-0.801580,-0.118948,-1.394653,-0.818344,-0.242766,-1.055114,-0.763770,-1.10630,-0.212666,-1.581549,-0.958182,-2.955365e-15,-2.407148,-2.256203,1.789347,-0.714046,2.289845,-0.120956,-0.27572,-0.070962,-0.092701,-0.507792,0.137231,85000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.161040,-0.415465,-0.324335,-0.820093,1.268065,-1.107260,-1.683097,0.856650,0.267880,-0.285382,-0.915384,-0.750599,-0.487113,0.988803,-0.118948,0.458779,1.104846,-0.242766,-1.055114,1.236417,0.13616,-0.212666,0.270639,2.141974,-1.603875e+00,-1.056091,-1.119276,-0.739841,-0.714046,3.900586,-0.120956,-0.27572,-0.070962,-0.092701,-0.133360,-0.618833,175000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.161040,-0.817120,-0.528808,-0.820093,0.372058,-1.531051,-1.683097,-0.599726,-0.496707,-0.285382,0.531863,-0.081174,-0.370736,0.711260,-0.118948,0.311953,-0.818344,-0.242766,0.773333,-0.763770,0.13616,-0.212666,-0.346757,0.591896,-2.210452e+00,-1.056091,-0.797147,-0.739841,-0.714046,3.546223,-0.120956,-0.27572,-0.070962,-0.092701,-0.507792,1.649359,127000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
dataset_test.head(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,GarageQual_TA,GarageQual_nan,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_nan,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_nan,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_nan,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MiscFeature_nan,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.895141,0.048970,-0.239805,-0.007126,2.236970,-0.335418,0.898547,-0.490825,0.514778,-0.300768,-0.372331,0.043937,-0.175554,-0.767463,-0.124911,-0.757240,-0.823714,3.785613,-0.915164,-0.750992,0.269063,-0.206665,-0.208943,-0.920683,-0.605702,-0.916092,-0.873322,0.985425,-0.699038,-0.391863,-0.100213,-0.247133,-0.058521,-0.155996,-1.477689,-1.334966,154500,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.070256,1.332825,0.412893,1.379981,-0.488462,0.761873,0.504885,1.228570,1.344641,-0.300768,-0.289759,0.981207,0.944414,1.877512,-0.124911,2.186756,1.115999,-0.234020,0.854557,1.189624,0.269063,-0.206665,1.621401,2.187287,0.656177,0.376173,1.124832,0.928880,-0.059667,-0.391863,-0.100213,-0.247133,-0.058521,-0.155996,-0.771494,1.623744,325000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.653792,-0.592958,-0.147047,-0.700679,0.420015,-1.609692,-1.709464,-0.490825,-0.976275,-0.300768,1.071489,-0.074382,-0.279255,-0.767463,-0.124911,-0.833019,-0.823714,-0.234020,-0.915164,-0.750992,-0.903568,-0.206665,-0.819058,0.633302,-2.071110,0.376173,-0.445146,-0.823990,-0.699038,1.880882,-0.100213,-0.247133,-0.058521,-0.155996,-1.124592,1.623744,115000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.171093,-0.868070,-0.819791,-0.007126,1.328493,-0.901762,-1.709464,-0.490825,-0.078944,-0.300768,0.120738,-0.083662,-0.341476,0.788405,-0.124911,0.371860,-0.823714,-0.234020,0.854557,-0.750992,0.269063,-0.206665,0.401171,2.187287,-1.134877,0.376173,-0.177536,-0.823990,-0.219510,0.237513,-0.100213,-0.247133,-0.058521,-0.155996,1.347092,-1.334966,159000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.895141,0.920158,0.521959,2.073534,-0.488462,1.222028,1.144586,-0.158346,1.322151,-0.300768,0.104224,1.345443,1.255517,-0.767463,-0.124911,0.288503,1.115999,-0.234020,0.854557,-0.750992,-0.903568,-0.206665,-0.208943,0.633302,1.226058,1.668438,2.016865,1.324690,-0.699038,-0.391863,-0.100213,-0.247133,-0.058521,-0.155996,0.993994,0.884067,315500,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
def standardize_data(dataset, exclude_columns,):
  dataset = dataset.copy(deep=True)
  dataset_cols = list(set(dataset.columns.values) - set(exclude_columns))
  for column_name in dataset_cols:
    column = dataset[column_name].to_numpy()
    # print(column, column_name)
    nan_indexes = np.isnan(column)
    real_indexes = ~np.isnan(column)

    n = real_indexes.astype(int).sum()
    real_column = column[real_indexes]

    real_column_mean = real_column.sum() / n # or .mean()  
    real_column_var = (np.sum([(column[i]-real_column_mean)**2 if item else 0 for i, item in enumerate(list(real_indexes))]) / n)**(1/2)
    std_column = np.array([(column[i]-real_column_mean) / real_column_var if item else 0 for i, item in enumerate(list(real_indexes))])
    
    if np.isnan(std_column).any():
      std_column = np.zeros_like(std_column)
    
    dataset.loc[:, column_name] = std_column 
  return dataset

# dataset_train = standardize_data(dataset_train, ['SalePrice'])
# dataset_test = standardize_data(dataset_test, ['SalePrice'])
# dataset_train.head()

In [ ]:
dataset_test.head()

In [ ]:

fig, ax = plt.subplots(figsize=(40,40))
corr = dataset_train.corr()# plot the heatmap
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True), ax=ax)

In [94]:
class MultiLayerModel(nn.Module):
  def __init__(self, 
                features=None,
                targets=None
    ):
    super(MultiLayerModel, self).__init__()
    self.lin_1 = nn.Linear(in_features=features, out_features=features*2, ) 
    self.lin_2 = nn.Linear(in_features=features*2, out_features=features) 
    self.lin_3 = nn.Linear(in_features=features, out_features=targets) 

    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()

  def forward(self, x):
    x = self.tanh(self.lin_1(x))
    x = self.tanh(self.lin_2(x))
    x = self.lin_3(x)
    return x

class RMSLELogLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()
      
  def forward(self, pred, actual):
    clipped_preds = torch.clamp(pred, 1, float('inf'))
    return torch.sqrt(self.mse(torch.log(clipped_preds), torch.log(actual)))

model = MultiLayerModel(features=3, targets=1)
print(model(torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [0.11, 0.12, 0.13]], dtype=torch.float32)))
predicts = model(torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [0.11, 0.12, 0.13]], dtype=torch.float32))
loss = RMSLELogLoss()(predicts, torch.tensor([[0.12], [0.15], [0.18], [0.19]]))
print(loss)
loss.backward()


tensor([[-0.5505],
        [-0.5455],
        [-0.5400],
        [-0.5421]], grad_fn=<AddmmBackward0>)
tensor(1.8570, grad_fn=<SqrtBackward0>)


In [ ]:
class CustomPandasDataset(Dataset):
  def __init__(self, 
                dataset=None,
                target_columns=None,
                infer=False
      ):
      self.dataset = dataset
      self.target_columns = target_columns
      self.infer = infer

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if self.infer:
        data_row = self.dataset.iloc[idx]
        target = torch.tensor([0], dtype=torch.float32)
        features = torch.tensor(data_row.values, dtype=torch.float32)
        return features, target 
      else:
        data_row = self.dataset.iloc[idx]
        target = torch.tensor(data_row[self.target_columns].values, dtype=torch.float32)
        data_row = data_row.drop(labels=self.target_columns)
        features = torch.tensor(data_row.values, dtype=torch.float32)

        return features, target 

temp_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=['SalePrice'],
    # infer=True
)

temp_dataloader = DataLoader(
    dataset=temp_dataset, 
    batch_size=16, 
    shuffle=True
)

next(iter(temp_dataloader))
# temp_dataset

In [97]:
class Trainer:
  def __init__(self, 
               model=None, 
               train_dataloader=None,
               test_dataloader=None,
               optimizer_class=None,
               loss_func=None,
    ): 
    self.model = model
    self.train_dataloader = train_dataloader
    self.test_dataloader = test_dataloader
    self.optimizer_class = optimizer_class
    self.optimizer = None
    self.loss_func = loss_func
    self.estimate_func = RMSLELogLoss()

    # self.unit_tests()

  def train(self, 
            epochs=1,
            lr=5
    ):
    self.optimizer = self.optimizer_class(
        params=self.model.parameters(),
        lr=lr
    )
    for epoch in range(epochs):
      print(f"Epoch {epoch}:")
      total_examples = 0
      total_loss = 0
      last_loss = 0
      for features, targets in self.train_dataloader:
        self.optimizer.zero_grad()
        predicts = self.model(features)
        # print(targets)
        loss = self.loss_func(predicts, targets)
        loss.backward()
        self.optimizer.step()
        with torch.no_grad():
          loss_num = self.estimate_func(predicts, targets)
          last_loss = loss_num 
          total_loss += loss_num 
          total_examples += len(targets)
        
      # print(f"train loss => {total_loss / total_examples}")
      print(f"train_loss => {last_loss}")
      
      total_examples = 0
      total_loss = 0
      total_acc = 0
      last_loss = 0
      with torch.no_grad():
        for features, targets in self.test_dataloader:
          predicts = self.model(features)
          loss = self.estimate_func(predicts, targets)
          last_loss = loss
          total_loss += loss
          # predicts = predicts.argmax(-1)
          total_examples += len(targets)
          # total_acc += (predicts == targets).type(torch.float32).sum()

      # print(f"test loss => {total_loss / total_examples}")
      print(f"test_loss => {last_loss}")
      # print(f"test acc => {total_acc / total_examples}")

  
  def unit_tests(self):
    print("run unit tests")
    self.one_batch_overfit()
    print("Ok")
  
  def one_batch_overfit(self, 
                        lr=0.01,
                        epochs=50
    ):
    self.optimizer = self.optimizer_class(
        params=self.model.parameters(),
        lr=lr,
        weight_decay=0
    )

    one_batch = next(iter(self.train_dataloader))
    for epoch in range(epochs):
      self.optimizer.zero_grad()
      # print(f"Epoch {epoch}:")
      features, targets = one_batch
      predicts = self.model(features)
      loss = self.loss_func(predicts, targets)
      loss.backward()
      self.optimizer.step()

      print(f"train loss => {loss/len(targets)}")
      

target_columns = ['SalePrice']
batch_size = 64
train_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=target_columns
)
train_dataloader = DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

test_dataset = CustomPandasDataset(
    dataset=dataset_test,
    target_columns=target_columns
)
test_dataloader = DataLoader(
    dataset=test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

# print(train_dataset[0])
features_size = len(train_dataset[0][0])
targets_size = 1 #len(train_dataset[0][1])
# model=MultiLayerModel(
#     features=features_size, 
#     targets=targets_size
# )

model = nn.Linear(in_features=features_size, out_features=targets_size)

# optimizer_class = CustomSGD
optimizer_class = torch.optim.Adam
# optimizer_class = torch.optim.SGD
# loss_func = RMSLELogLoss()
loss_func = nn.MSELoss()
# loss_func = LinearRegressionDirty.cross_entropy
trainer = Trainer(
  model=model,
  train_dataloader=train_dataloader,
  test_dataloader=test_dataloader,
  optimizer_class=optimizer_class,
  loss_func=loss_func
)
trainer.train(epochs=100)

Epoch 0:
train_loss => 3.7389674186706543
test_loss => 3.737757921218872
Epoch 1:
train_loss => 3.0212512016296387
test_loss => 3.05046010017395
Epoch 2:
train_loss => 2.611771821975708
test_loss => 2.6510326862335205
Epoch 3:
train_loss => 2.3254470825195312
test_loss => 2.3697025775909424
Epoch 4:
train_loss => 2.105870008468628
test_loss => 2.153118133544922
Epoch 5:
train_loss => 1.9282475709915161
test_loss => 1.9774986505508423
Epoch 6:
train_loss => 1.779468059539795
test_loss => 1.830161213874817
Epoch 7:
train_loss => 1.6517530679702759
test_loss => 1.7035399675369263
Epoch 8:
train_loss => 1.5401076078414917
test_loss => 1.592757225036621
Epoch 9:
train_loss => 1.4411330223083496
test_loss => 1.4944839477539062
Epoch 10:
train_loss => 1.3524117469787598
test_loss => 1.4063467979431152
Epoch 11:
train_loss => 1.2721633911132812
test_loss => 1.3265948295593262
Epoch 12:
train_loss => 1.1990385055541992
test_loss => 1.2538992166519165
Epoch 13:
train_loss => 1.1319897174835205
t

In [ ]:
# model=MultiLayerModel(
#     features=features_size, 
#     targets=targets_size
# )
# model = nn.Linear(in_features=features_size, out_features=targets_size)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# loss_func = nn.MSELoss()
# acc_func = RMSLELogLoss()

# for ep in range(15):
#   total_loss = 0
#   total_elements = 0
#   for fea, tar in train_dataloader:
#     pred = model(fea)
#     loss = loss_func(pred, tar)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
#     with torch.no_grad():
#       total_loss += acc_func(pred, tar)
#       total_elements += len(tar)
#   print(total_loss/total_elements)

## submit

In [111]:
dataset_submit = pd.read_csv("./data/HousePricesKaggle/test.csv")


drop_columns = 'Id'.split()
dataset_submit = dataset_submit.drop(columns=drop_columns)

select_columns = dataset_submit.dtypes[dataset_submit.dtypes != 'object'].index.tolist()
# select_columns = list(set(select_columns) - set(['SalePrice']))

dataset_submit[select_columns] = dataset_submit[select_columns].apply(
    lambda x: (x - x.mean()) / (x.std()))
target_columns = ['SalePrice']
dataset_submit[select_columns] = dataset_submit[select_columns].fillna(dataset_submit[select_columns].mean() / dataset_submit[select_columns].std())
categorical_labels = dataset_submit.dtypes[dataset_submit.dtypes == 'object'].index.tolist()

dataset_submit = encode_categorical(dataset_submit, categorical_labels, dataset)

batch_size = len(dataset_submit)
submit_dataset = CustomPandasDataset(
    dataset=dataset_submit,
    target_columns=target_columns,
    infer=True
)

submit_dataloader = DataLoader(
    dataset=submit_dataset, 
    batch_size=batch_size, 
    shuffle=False,
)



In [ ]:
dataset_submit

In [113]:
with torch.no_grad():
  features, targets = next(iter(submit_dataloader))
  # print(features)
  preds = trainer.model(features).numpy().reshape(-1)
  # print(preds)

  test_data_subm = pd.read_csv("./data/HousePricesKaggle/test.csv")
  test_data_subm = test_data_subm[['Id']]
  test_data_subm['SalePrice'] = preds

  test_data_subm.to_csv('./data/HousePricesKaggle/submission_my.csv', index=False)

In [ ]:
np.array([[1], [2], [3]]).reshape(-1)

array([1, 2, 3])